# Διαδικασία: 

Αρχικά, θα δοκιμάσουμε να υλοποιήσουμε "Kasiski Attack" για να μπορέσουμε να βρούμε (κατά πάσα πιθανότητα) το μήκος 
keylength του κλειδιού, βρίσκοντας επαναλαμβανόμενα patterns στο encrypted μήνυμα.

Έπειτα, μόλις βρούμε το μήκος του κλειδιού πρέπει να βρούμε την λέξη, πιθανόν με το "Τεστ του Friedman" και τη χρήση
του δείκτη σύμπτωσης, χωρίζοντας το encrypted μήνυμα σε keylength στήλες. 

Τέλος, βρίσκουμε εύκολα αφού έχουμε το κλειδί χρησιμοποιώντας την μέθοδο του %26 για να βρούμε το αρχικό μήνυμα.


In [1]:
# Imports
import pandas as pd
import math
from functools import reduce
import operator

In [2]:
with open('vigenere.txt', 'r', encoding='utf-8-sig') as file:
    coded_msg = file.read().replace('\n', '')
display(coded_msg)

'MYHSIFPFGIMUCEXIPRKHFFQPRVAGIDDVKVRXECSKAPFGHMESJWUSSEHNEZIXFFLPQDVTCEUGTEEMFRQXWYCLPPAMBSKSTTPGSMIDNSESZJBDWJWSPQYINUVRFXPVPCEOZQRBNLUIINSRPXLEEHKSTTPGCEIMCSKVVVTJQRBSIUCKJOIIXXOVHYEFLINOEXFDPZJVFKTETVFXTTVJVRTBXRVGJRAIFPSRGTDXYSIWYXWVFPAQSSEHNEZIXFVRXQPRURVWBXWVCEIMCSKVVVUCXYWJAAGPUHYIDTMJFFSYUSISMIDNSESRRPILVUFSPTEIHYMEGMTVRRPREEDISHXHVTFVQKIIMFRQILVKRCAUPZTVGMCFVTIIQPRUPVEGIMWICFGIAVVRZQASJHKLQLEPUIIQSLRGGSUHSESUQQCWJCLPEWEJPRVDXGRRVHFWINCIPPLMKVYEFTLRGXSAHIJHVTBTHLGZRFDQZGVVKPRUPCSASWYSUAQWEMSUIHTPFDVHEEIVRSYITLRJVWTJXFIIWQAZVGZRYPGYWEIDNXYOKKUKIJOSYZSEEQVLMHPVTKYEXRNOEXAJVBBFAXTHXSYEEBEUSLWONRZQRPAJVTZVZQGRVGJLMGHRBUYZZMERNIFWMEYKSABYTVRRPUIVZKSAAMKHCIYDVVHYEZBETVZRQGCNSEIQSLLARRUICDCIIFWEEQCIHTVESJWITRVSUOUCHESJWMCHXSEXXTRVGJAUILFIKXTTWVELEXXXZSJPUUINWCPNTZZCCIZIEERRPXLMCZSIXDWKHYIMTVFDCEZTEERKLQGEUWFLMKISFFYSWXLGTPAHIIHFKQILVFKLQKIIMEEFJVVCWXTTWVWEZQCXZCEWOGMVGFYFUSIHYISDSUBVWEXRDSEGDXIJCLXRDVLBZZQGWRZSVAILVFYSASJFFKLQJRZHPSRJWRZCIHTRECNQKKSZQVMEGIRQYMZVQZZCMACWKVISGVLFIKXTTAF

In [3]:
len(coded_msg)

2597

In [4]:
def kasiski_attack(text, n):
    i = 0
    count = {}
    
    for i in range(0, len(text)-(n-1)):
        #s = text[i] + text[i+1] + text[i+2] + text[i+3]
        s = text[i]
        for j in range(i+1, i+n):
            if j < len(text):
                s = s + text[j]
        if s in count:
            count[s] += 1
        else:
            count[s] = 1
        
    return count

In [5]:
columns = ['Length','Substring', 'Frequency']
df = pd.DataFrame(columns = columns)

In [6]:
for q in range(4,10):
    count = kasiski_attack(coded_msg, q)
    for key in count:
        if count[key] > 2:
            df = df.append({'Length': q, 'Substring': key, 'Frequency': count[key]}, ignore_index=True)
            #print ("Length:", q, " substring:", key, " freq:", count[key])

In [7]:
la = [[] for _ in range(52)]
for index, row in df.iterrows():
    count_stars = 0
    count = 0
    start = False
    for char in coded_msg.replace(row['Substring'], "*"):
        if char == "*":
            if count != 0:
                la[index].append(count-1)
            count_stars = count_stars + 1
            start = True
            count=0
        if count_stars == row['Frequency']:
            break
        if start == True:
            count = count + 1

In [8]:
df['Distances'] = la 

In [9]:
def find_gcd(list):
    x = reduce(math.gcd, list)
    return x

In [10]:
gcd_list = []
for index, row in df.iterrows():
    gcd_list.append(find_gcd(row['Distances']))

In [11]:
df['GCD'] = gcd_list 

In [12]:
display(df)

,Length,Substring,Frequency,Distances,GCD
0,4,UCEX,3,"[1354, 381]",1
1,4,KVRX,5,"[1662, 381, 52, 24]",1
2,4,MESJ,3,"[1886, 612]",2
3,4,ESJW,4,"[661, 10, 1823]",1
4,4,SKVV,3,"[108, 1039]",1
5,4,RVGJ,3,"[402, 115]",1
6,4,VRXQ,3,"[983, 458]",1
7,4,QPRU,3,"[115, 892]",1
8,4,UHYI,4,"[899, 80, 458]",1
9,4,SRRP,3,"[1011, 1193]",1


In [13]:
indexNames = df[df['GCD'] < 3 ].index

df.drop(indexNames , inplace=True)
df.reset_index()

,index,Length,Substring,Frequency,Distances,GCD
0,10,4,QKII,3,"[500, 1445]",5
1,16,4,FKLQ,3,"[87, 339]",3
2,32,5,IKXTT,3,"[240, 1332]",12
3,33,5,KLQJR,3,"[765, 9]",9
4,46,7,KVVTQDT,4,"[14, 574, 441]",7
5,47,7,VVTQDTC,4,"[14, 574, 441]",7
6,48,7,VTQDTCS,4,"[14, 574, 441]",7


In [14]:
n=7
#https://stackoverflow.com/questions/22571259/split-a-string-into-n-equal-parts
parts = [coded_msg[i:i+n] for i in range(0, len(coded_msg), n)]

In [15]:
display(parts)

['MYHSIFP',
 'FGIMUCE',
 'XIPRKHF',
 'FQPRVAG',
 'IDDVKVR',
 'XECSKAP',
 'FGHMESJ',
 'WUSSEHN',
 'EZIXFFL',
 'PQDVTCE',
 'UGTEEMF',
 'RQXWYCL',
 'PPAMBSK',
 'STTPGSM',
 'IDNSESZ',
 'JBDWJWS',
 'PQYINUV',
 'RFXPVPC',
 'EOZQRBN',
 'LUIINSR',
 'PXLEEHK',
 'STTPGCE',
 'IMCSKVV',
 'VTJQRBS',
 'IUCKJOI',
 'IXXOVHY',
 'EFLINOE',
 'XFDPZJV',
 'FKTETVF',
 'XTTVJVR',
 'TBXRVGJ',
 'RAIFPSR',
 'GTDXYSI',
 'WYXWVFP',
 'AQSSEHN',
 'EZIXFVR',
 'XQPRURV',
 'WBXWVCE',
 'IMCSKVV',
 'VUCXYWJ',
 'AAGPUHY',
 'IDTMJFF',
 'SYUSISM',
 'IDNSESR',
 'RPILVUF',
 'SPTEIHY',
 'MEGMTVR',
 'RPREEDI',
 'SHXHVTF',
 'VQKIIMF',
 'RQILVKR',
 'CAUPZTV',
 'GMCFVTI',
 'IQPRUPV',
 'EGIMWIC',
 'FGIAVVR',
 'ZQASJHK',
 'LQLEPUI',
 'IQSLRGG',
 'SUHSESU',
 'QQCWJCL',
 'PEWEJPR',
 'VDXGRRV',
 'HFWINCI',
 'PPLMKVY',
 'EFTLRGX',
 'SAHIJHV',
 'TBTHLGZ',
 'RFDQZGV',
 'VKPRUPC',
 'SASWYSU',
 'AQWEMSU',
 'IHTPFDV',
 'HEEIVRS',
 'YITLRJV',
 'WTJXFII',
 'WQAZVGZ',
 'RYPGYWE',
 'IDNXYOK',
 'KUKIJOS',
 'YZSEEQV',
 'LMHPVTK',
 'YEXRNOE',
 'XA

In [16]:
mapping = [list(word) for word in parts]

In [17]:
display(mapping)

[['M', 'Y', 'H', 'S', 'I', 'F', 'P'],
 ['F', 'G', 'I', 'M', 'U', 'C', 'E'],
 ['X', 'I', 'P', 'R', 'K', 'H', 'F'],
 ['F', 'Q', 'P', 'R', 'V', 'A', 'G'],
 ['I', 'D', 'D', 'V', 'K', 'V', 'R'],
 ['X', 'E', 'C', 'S', 'K', 'A', 'P'],
 ['F', 'G', 'H', 'M', 'E', 'S', 'J'],
 ['W', 'U', 'S', 'S', 'E', 'H', 'N'],
 ['E', 'Z', 'I', 'X', 'F', 'F', 'L'],
 ['P', 'Q', 'D', 'V', 'T', 'C', 'E'],
 ['U', 'G', 'T', 'E', 'E', 'M', 'F'],
 ['R', 'Q', 'X', 'W', 'Y', 'C', 'L'],
 ['P', 'P', 'A', 'M', 'B', 'S', 'K'],
 ['S', 'T', 'T', 'P', 'G', 'S', 'M'],
 ['I', 'D', 'N', 'S', 'E', 'S', 'Z'],
 ['J', 'B', 'D', 'W', 'J', 'W', 'S'],
 ['P', 'Q', 'Y', 'I', 'N', 'U', 'V'],
 ['R', 'F', 'X', 'P', 'V', 'P', 'C'],
 ['E', 'O', 'Z', 'Q', 'R', 'B', 'N'],
 ['L', 'U', 'I', 'I', 'N', 'S', 'R'],
 ['P', 'X', 'L', 'E', 'E', 'H', 'K'],
 ['S', 'T', 'T', 'P', 'G', 'C', 'E'],
 ['I', 'M', 'C', 'S', 'K', 'V', 'V'],
 ['V', 'T', 'J', 'Q', 'R', 'B', 'S'],
 ['I', 'U', 'C', 'K', 'J', 'O', 'I'],
 ['I', 'X', 'X', 'O', 'V', 'H', 'Y'],
 ['E', 'F', 

In [20]:
letters = []
for i in range(0,len(mapping)):
    letters.append(mapping[i][0])
text = ''.join(map(str, letters)) 
display(text)

'MFXFIXFWEPURPSIJPRELPSIVIIEXFXTRGWAEXWIVAISIRSMRSVRCGIEFZLISQPVHPESTRVSAIHYWWRIKYLYXAEWPZLYISRSIERIRIIWOWXAXEPPIPIIELLYPKLEXZWYIWGXZVSLSIQMMMSXHRTQRXHXWLWXMRRIILWXVIMSYWKIRIECRWXAWXMILJTMVTRKMXIMMXYWISIPAQVILALAIVHXYXCRHKVXYEGRGREIYQWRPILPYMIYXLILIIVMXSVJVILVOVLSMRSSTYQYTESTGEXVXRXPLXLJFYIJIZXLJEWXIYXXAHSXZLAXZEEKVLWWWLIIIIJLMRPXIPXPTSJJLMWSISAMVSAIERRWSHSHGRIPPEWVRICP'

In [23]:
df_list = []
columns = ['Letter', 'Frequency']
df_letters = pd.DataFrame(columns = columns)
for j in range(0,n):
    letters = []
    for i in range(0,len(mapping)):
        letters.append(mapping[i][j])
    text = ''.join(map(str, letters))
    print(str(j))
    print(text)
    count = kasiski_attack(text, 1)
    #print(count)
    df_list.append(count)

0
MFXFIXFWEPURPSIJPRELPSIVIIEXFXTRGWAEXWIVAISIRSMRSVRCGIEFZLISQPVHPESTRVSAIHYWWRIKYLYXAEWPZLYISRSIERIRIIWOWXAXEPPIPIIELLYPKLEXZWYIWGXZVSLSIQMMMSXHRTQRXHXWLWXMRRIILWXVIMSYWKIRIECRWXAWXMILJTMVTRKMXIMMXYWISIPAQVILALAIVHXYXCRHKVXYEGRGREIYQWRPILPYMIYXLILIIVMXSVJVILVOVLSMRSSTYQYTESTGEXVXRXPLXLJFYIJIZXLJEWXIYXXAHSXZLAXZEEKVLWWWLIIIIJLMRPXIPXPTSJJLMWSISAMVSAIERRWSHSHGRIPPEWVRICP
1
YGIQDEGUZQGQPTDBQFOUXTMTUXFFKTBATYQZQBMUADYDPPEPHQQAMQGGQQQUQEDFPFABFKAQHEITQYDUZMEAXEOAQMZFAPAYZQQUFHIUMXUTXUNZXXMZQMSAQQFTQOFSEDRQAAQRHKEZAGTYEMQMXDUKMYGYUFFIQMPFDEZFEDNQZDABEQUMMQBQDQXZQPMQKHEQSDFEPKMTQXXMTUTQUUDXFAZQUEAZXTITPOMMMKAQYMAYZMPQQPQPGMQRDKAFEOAQUQYEYZDAFQKQHIADOTFQQTQQAUDQXFQDQTQPOGBFZGRIQDIQMAIQRZQUQQQNMZDKKGMEQXAFHTMQIUGMERRIPUADPUPFFORDAZIUPEQXBEEMYXX
2
HIPPDCHSIDTXATNDYXZILTCJCXLDTTXIDXSIPXCCGTUNITGRXKIUCPIIALSHCWXWLTHTDPSWTETJAPNKSHXJTBNJGGZWBUADBGSIWTTCCTITXUTILDTTGKWHIKJTCGUDXXDGISJJTKGVCVTXDXCCTTBHITGECEDWPNTWNCJIXTXHLIUGIDACIPDNDDAIDHHALTWGXHDBTDKDCXAIPCPAATTXAJDGKTICBXXXHWGGRDINTKKPNGDDJWJPCA

In [24]:
for i in range(0, n):
    print(i)
    print(df_list[i])

0
{'M': 18, 'F': 6, 'X': 36, 'I': 52, 'W': 25, 'E': 21, 'P': 21, 'U': 1, 'R': 29, 'S': 27, 'J': 9, 'L': 26, 'V': 20, 'T': 9, 'G': 7, 'A': 13, 'C': 4, 'Z': 7, 'Q': 6, 'H': 9, 'Y': 17, 'K': 6, 'O': 2}
1
{'Y': 13, 'G': 12, 'I': 12, 'Q': 64, 'D': 23, 'E': 22, 'U': 24, 'Z': 19, 'P': 18, 'T': 18, 'B': 8, 'F': 24, 'O': 8, 'X': 18, 'M': 29, 'K': 11, 'A': 27, 'H': 7, 'N': 3, 'S': 3, 'R': 8}
2
{'H': 18, 'I': 32, 'P': 20, 'D': 39, 'C': 25, 'S': 10, 'T': 51, 'X': 32, 'A': 17, 'N': 12, 'Y': 1, 'Z': 3, 'L': 10, 'J': 14, 'G': 22, 'U': 11, 'R': 3, 'K': 12, 'W': 18, 'E': 7, 'B': 10, 'V': 3, 'Q': 1}
3
{'S': 28, 'M': 26, 'R': 36, 'V': 23, 'X': 30, 'E': 30, 'W': 27, 'P': 22, 'I': 43, 'Q': 10, 'K': 5, 'O': 4, 'F': 3, 'L': 21, 'H': 12, 'A': 10, 'G': 7, 'Z': 5, 'Y': 8, 'C': 5, 'J': 8, 'T': 8}
4
{'I': 19, 'U': 16, 'K': 38, 'V': 56, 'E': 35, 'F': 31, 'T': 9, 'Y': 16, 'B': 3, 'G': 9, 'J': 14, 'N': 11, 'R': 22, 'Z': 25, 'P': 9, 'W': 3, 'L': 12, 'M': 4, 'X': 7, 'D': 8, 'C': 20, 'S': 4}
5
{'F': 21, 'C': 43, 'H': 4

In [25]:
for i in range(0, n):
    temp = max(df_list[i].items(), key=operator.itemgetter(1))[0]
    print(temp + str(': ')+str(df_list[i][temp]))

I: 52
Q: 64
T: 51
I: 43
V: 56
S: 47
V: 49


In [72]:
data0 = []
data1 = []
data2 = []
data3 = []
data4 = []
data5 = []
data6 = []

In [71]:
p=0
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data0.append(values)

In [74]:
p=1
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data1.append(values)

In [76]:
p=2
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data2.append(values)

In [79]:
p=3
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data3.append(values)

In [81]:
p=4
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data4.append(values)

In [84]:
p=5
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data5.append(values)

In [85]:
p=6
sortednames=sorted(df_list[p].keys(), key=lambda x:x.lower())
for i in sortednames:
    values=df_list[p][i]
    #print(i + ": " +str(values))
    data6.append(values)

In [35]:
data = {
    "A": 8.12,
    "B": 1.49,
    "C": 2.71,
    "D": 4.32,
    "E": 12.02,
    "F": 2.3,
    "G": 2.03,
    "H": 5.92,
    "I": 7.31,
    "J": 0.1,
    "K": 0.69,
    "L": 3.92,
    "M": 2.61,
    "N": 6.95,
    "O": 7.68,
    "P": 1.82,
    "Q": 0.11,
    "R": 6.02,
    "S": 6.28,
    "T": 9.1,
    "U": 2.88,
    "V": 1.11,
    "W": 2.09,
    "X": 0.17,
    "Y": 2.11,
    "Z": 0.07

}

In [87]:
data0 = [13,0,4,0,21,6,8,9,52,9,6,26,18,0,2,21,6,29,27,9,1,20,25,36,17,7]
data1 = [27,8,0,23,22,24,12,7,12,0,11,0,29,3,8,18,64,8,3,18,24,0,0,18,13,19]
data2 = [17,10,52,39,7,0,22,18,32,14,12,10,0,12,0,20,1,3,10,51,11,3,18,32,1,3]
data3 = [10,0,5,0,30,3,7,12,43,8,5,21,26,0,4,22,10,36,28,8,0,23,27,30,8,5]
data4 = [0,3,20,8,35,31,9,0,19,14,38,12,4,11,0,9,0,22,4,9,16,56,3,7,16,25]
data5 = [11,24,43,6,1,21,21,40,10,4,5,0,10,0,18,6,10,12,47,10,9,28,16,0,2,17]
data6 = [0,2,19,5,21,25,5,0,18,23,40,21,4,10,0,8,0,37,8,5,19,49,6,3,26,17]
data = [8.12, 1.49, 2.71, 4.32, 12.02, 2.3, 2.03, 5.92, 7.31, 0.1, 0.69, 3.92,
        2.61, 6.95, 7.68, 1.82, 0.11, 6.02, 6.28, 9.1, 2.88, 1.11, 2.09, 0.17, 2.11, 0.07]

In [89]:
#display(data)

In [90]:
import string
column = list(string.ascii_uppercase)
new = pd.DataFrame(columns = column)

In [94]:
newdf=pd.DataFrame(fulldata,columns=column)

In [92]:
fulldata = []
fulldata.append(data)
fulldata.append(data0)
fulldata.append(data1)
fulldata.append(data2)
fulldata.append(data3)
fulldata.append(data4)
fulldata.append(data5)
fulldata.append(data6)

In [93]:
print(len(fulldata[0]))
print(len(fulldata[1]))
print(len(fulldata[2]))
print(len(fulldata[3]))
print(len(fulldata[4]))
print(len(fulldata[5]))
print(len(fulldata[6]))
print(len(fulldata[7]))

26
26
26
26
26
26
26
26


In [96]:
display(newdf)

,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
0,8.12,1.49,2.71,4.32,12.02,2.3,2.03,5.92,7.31,0.1,...,0.11,6.02,6.28,9.1,2.88,1.11,2.09,0.17,2.11,0.07
1,13.00,0.00,4.00,0.00,21.00,6.0,8.00,9.00,52.00,9.0,...,6.00,29.00,27.00,9.0,1.00,20.00,25.00,36.00,17.00,7.00
2,27.00,8.00,0.00,23.00,22.00,24.0,12.00,7.00,12.00,0.0,...,64.00,8.00,3.00,18.0,24.00,0.00,0.00,18.00,13.00,19.00
3,17.00,10.00,52.00,39.00,7.00,0.0,22.00,18.00,32.00,14.0,...,1.00,3.00,10.00,51.0,11.00,3.00,18.00,32.00,1.00,3.00
4,10.00,0.00,5.00,0.00,30.00,3.0,7.00,12.00,43.00,8.0,...,10.00,36.00,28.00,8.0,0.00,23.00,27.00,30.00,8.00,5.00
5,0.00,3.00,20.00,8.00,35.00,31.0,9.00,0.00,19.00,14.0,...,0.00,22.00,4.00,9.0,16.00,56.00,3.00,7.00,16.00,25.00
6,11.00,24.00,43.00,6.00,1.00,21.0,21.00,40.00,10.00,4.0,...,10.00,12.00,47.00,10.0,9.00,28.00,16.00,0.00,2.00,17.00
7,0.00,2.00,19.00,5.00,21.00,25.0,5.00,0.00,18.00,23.0,...,0.00,37.00,8.00,5.0,19.00,49.00,6.00,3.00,26.00,17.00


In [99]:
newdf.to_csv('freq2.csv', index=False)